# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

J. Li  ->  J. Li  |  ['J. Li']
M. Benisty  ->  M. Benisty  |  ['M. Benisty']
J. Li  ->  J. Li  |  ['J. Li']
F. Walter  ->  F. Walter  |  ['F. Walter']
T. Henning  ->  T. Henning  |  ['T. Henning']


K. El-Badry  ->  K. El-Badry  |  ['K. El-Badry']
Arxiv has 72 new papers today
          5 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/5 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2412.14246


extracting tarball to tmp_2412.14246...

 done.
Retrieving document from  https://arxiv.org/e-print/2412.14402


/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2412.14246/body.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'author' from 'tmp_2412.14246/author.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


...

extracting tarball to tmp_2412.14707...

 done.


Found 122 bibliographic references in tmp_2412.14707/ngc6240_gto.bbl.
Issues with the citations
syntax error in line 18: '=' expected
Retrieving document from  https://arxiv.org/e-print/2412.15153


extracting tarball to tmp_2412.15153...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2412.14707-b31b1b.svg)](https://arxiv.org/abs/2412.14707) | **MICONIC: Dual active galactic nuclei, star formation, and ionised gas outflows in NGC 6240 seen with MIRI/JWST**  |
|| L. H. Muñoz, et al. -- incl., <mark>F. Walter</mark>, <mark>T. Henning</mark> |
|*Appeared on*| *2024-12-20*|
|*Comments*| *23 pages, 10 figures. Accepted for publication in A&A on 26/11/2024*|
|**Abstract**|            As part of the guaranteed time observations program Mid-Infrared Characterization Of Nearby Iconic galaxy Centers (MICONIC), we used the medium-resolution spectrometer (MRS) of the Mid-Infrared Instrument (MIRI) on board of the JWST to study the nearby merger NGC6240. We aim to characterise the dual active galactic nuclei (AGN), the ionised gas outflows and the main properties of the interstellar medium over a mapped area of 6.6"x7.7". We obtained integral field spectroscopic mid-infrared data of NGC6240, resolving both nuclei for the first time in the full 5-28{\mu}m spectral range. We modelled the emission lines through a kinematic decomposition, finding that the fine-structure lines in the southern (S) nucleus are broader than for the northern (N) nucleus (full width at half maximum of $\geq$1500 vs ~700 km s^{-1} on average). High excitation lines, such as [NeV], [NeVI], and [MgV], are clearly detected in the N nucleus. In the S nucleus, the same lines can be detected but only after a decomposition of the PAH features in the integrated spectrum, due to a combination of a strong mid-IR continuum, the broad emission lines, and the intense star formation (SF). The SF is distributed all over the mapped FoV of 3.5kpc x 4.1kpc (projected), with the maximum located around the S nucleus. Both nuclear regions appear to be connected by a bridge region detected with all the emission lines. Based on the observed MRS line ratios and the high velocity dispersion ({\sigma}~600 km s^{-1}), shocks are also dominating the emission in this system. We detected the presence of outflows as a bubble north-west from the N nucleus and at the S nucleus. We estimated a ionised mass outflow rate of 1.4$\pm$0.3 M yr^{-1} and 1.8$\pm$0.2 M yr^{-1}, respectively. Given the derived kinetic power of these outflows, both the AGN and the starburst could have triggered them. [Abridged]         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2412.14246-b31b1b.svg)](https://arxiv.org/abs/2412.14246) | **Discovery of dual "little red dots" indicates excess clustering on kilo-parsec scales**  |
|| T. S. Tanaka, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2024-12-20*|
|*Comments*| *12 pages, 5 figures, and 1 table. Comments are welcome*|
|**Abstract**|            ``Little Red Dots'' (LRDs) are an abundant high-redshift population newly discovered by the James Webb Space Telescope (JWST). They are characterized by a red color in the rest-frame optical band, compact morphology, and broad Balmer emission lines (${\rm FWHM} \gtrsim 1000~{\rm km\,s^{-1}}$) that suggest an AGN nature. Using a method of pixel-by-pixel color selection and relaxing the compactness criteria, we identify three of the first dual LRD candidates in the COSMOS-Web survey with projected separations of $0.\!\!^{\prime\prime}2-0.\!\!^{\prime\prime}4$ (1-2 pkpc at their photometric redshifts). A comparison between existing LRD samples and mock data reveals that the projected separations of these dual LRD candidates are unlikely to result from chance projections of objects at different redshifts. In one case (CW-B5-15958), the dual LRD includes two bright sources ($m_{\rm F444W}=24.3$ and $24.8$) with characteristic V-shape spectral energy distribution (SEDs) and photometric redshifts consistent with each other. We find that CW-B5-15958 has a faint off-centered component and a companion galaxy. In the other two dual systems, the brighter LRD exhibits a V-shape SED, while the fainter LRD ($m_{\rm F444W}\gtrsim26$) is undetected in both F115W and F150W. These discoveries suggest that the angular auto-correlation function (ACF) of LRDs exhibits a significant excess ($\sim3\times10^2$ times) on sub-arcsec (kilo-parsec) separations compared to the extrapolation of a power-law ACF of JWST-found AGNs measured over $10^{\prime\prime}-100^{\prime\prime}$. Follow-up spectroscopic confirmation of their redshifts and the construction of a larger sample are essential to advance our understanding of the evolution of supermassive black holes and the importance of mergers in the early universe.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2412.14621-b31b1b.svg)](https://arxiv.org/abs/2412.14621) | **Diversity and universality: evolution of dwarf galaxies with self-interacting dark matter**  |
|| Z. C. Zeng, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2024-12-20*|
|*Comments*| *35 pages, 21 figures, happy holidays*|
|**Abstract**|            Dark matter halos with self-interacting dark matter (SIDM) experience a unique evolutionary phenomenon, in that their central regions eventually collapse to high density through the runaway gravothermal process after initially forming a large and low-density core. When coupled with orbital evolution, this is expected to naturally produce a large diversity in dark-matter halos' inner mass distribution, potentially explaining the diversity problem of dwarf galaxies. However, it remains unknown how the diversity in SIDM dark-matter halos propagates to the more easily observed luminous matter at the center of the halo, especially the stellar component. In this work, we use idealized N-body simulations with two species of particles (dark matter and stars) to study the response of the stellar properties of field and satellite dwarf galaxies to SIDM evolution and orbital effects on their halos. Galaxies' stellar components, including galaxy size, mass-to-light ratio, and stellar velocity dispersion, display a much larger scatter in SIDM than the standard cold dark matter (CDM) model. Importantly, we find signs of universality in the evolution pathways, or ``tidal tracks'', of SIDM dwarf satellites, which are physically interpretable and potentially parameterizable. This type of tidal-track model can be layered onto larger-scale, cosmological simulations to reconstruct the evolution of populations of SIDM dwarfs in cases where high-resolution simulations of galaxies are otherwise prohibitively expensive.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2412.15153-b31b1b.svg)](https://arxiv.org/abs/2412.15153) | **Cyclotron emitting magnetic white dwarfs in post common envelope binaries discovered with the Zwicky Transient Facility**  |
|| J. v. Roestel, et al. -- incl., <mark>K. El-Badry</mark> |
|*Appeared on*| *2024-12-20*|
|*Comments*| **|
|**Abstract**|            We present the discovery of 14 new (and recovery of 4 known) low accretion rate magnetic white dwarfs in post-common envelope binaries that emit strong cyclotron emission using the Zwicky Transient Facility (ZTF) light curves, doubling the known sample size. In addition, we discovered a candidate magnetic period bouncer and recovered three known ones. We confirmed the presence of cyclotron emission using low-resolution spectra in 19 objects. Using the ZTF light curves, follow-up spectra, and the spectral energy distribution, we measured the orbital period, magnetic field strength, and white dwarf temperature of each system. Although the phase-folded light curves have diverse shapes and show a much larger variability amplitude, we show that their intrinsic properties (e.g. period distribution, magnetic field strength) are similar to those of previously known systems. The diversity in light curve shapes can be explained by differences in the optical depth of the accretion spot and geometric differences, the inclination angle and the magnetic spot latitude. The evolutionary states of the longer period binaries are somewhat uncertain but are vary; we found systems consistent with being pre-polars, detached polars, or low-state polars. In addition, we discovered two new low-state polars that likely have brown dwarf companions and could be magnetic period bouncers.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2412.14402-b31b1b.svg)](https://arxiv.org/abs/2412.14402) | **Disk Evolution Study Through Imaging of Nearby Young Stars (DESTINYS): Dynamical Evidence of a Spiral-Arm-Driving and Gap-Opening Protoplanet from SAO 206462 Spiral Motion**  |
|| C. Xie, et al. -- incl., <mark>M. Benisty</mark> |
|*Appeared on*| *2024-12-20*|
|*Comments*| *11 pages, 3 figures. Invited paper accepted to special issue (this https URL) of Universe. Data in ancillary folder*|
|**Abstract**|            In the early stages of planetary system formation, young exoplanets gravitationally interact with their surrounding environments and leave observable signatures on protoplanetary disks. Among these structures, a pair of nearly symmetric spiral arms can be driven by a giant protoplanet. For the double-spiraled SAO 206462 protoplanetary disk, we obtained three epochs of observations spanning 7 yr using the Very Large Telescope's SPHERE instrument in near-infrared $J$-band polarized light. By jointly measuring the motion of the two spirals at three epochs, we obtained a rotation rate of $-0.85^\circ\pm0.05^\circ~{\rm yr}^{-1}$. This rate corresponds to a protoplanet at $66\pm3$ au on a circular orbit dynamically driving both spirals. The derived location agrees with the gap in ALMA dust-continuum observations, indicating that the spiral driver may also carve the observed gap. What is more, a dust filament at $\sim$63 au observed by ALMA coincides with the predicted orbit of the spiral-arm-driving protoplanet. This double-spiraled system is an ideal target for protoplanet imaging.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error list index out of range</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2412.14707/./figures/v_sigma_neii_1comp.png', 'tmp_2412.14707/./figures/v_sigma_neiii_1comp.png', 'tmp_2412.14707/./figures/v_sigma_feii_1comp.png', 'tmp_2412.14707/./figures/v_sigma_h1_65_1comp.png', 'tmp_2412.14707/./figures/regions_NGC6240_v2.png', 'tmp_2412.14707/./figures/v_sigma_oiv_1comp.png', 'tmp_2412.14707/./figures/v_sigma_nev_1comp_isolated.png', 'tmp_2412.14707/./figures/v_sigma_nevi_1comp.png', 'tmp_2412.14707/./figures/v_sigma_mgv_1comp.png']
copying  tmp_2412.14707/./figures/v_sigma_neii_1comp.png to _build/html/
copying  tmp_2412.14707/./figures/v_sigma_neiii_1comp.png to _build/html/
copying  tmp_2412.14707/./figures/v_sigma_feii_1comp.png to _build/html/
copying  tmp_2412.14707/./figures/v_sigma_h1_65_1comp.png to _build/html/
copying  tmp_2412.14707/./figures/regions_NGC6240_v2.png to _build/html/
copying  tmp_2412.14707/./figures/v_sigma_oiv_1comp.png to _build/html/
copying  tmp_2412.14707/./figures/v_sigma_nev_1comp_isolated.png to _build/html

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$</div>



<div id="title">

# MICONIC: dual AGN, star formation, and ionised gas outflows in NGC 6240 seen with MIRI/JWST

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2412.14707-b31b1b.svg)](https://arxiv.org/abs/2412.14707)<mark>Appeared on: 2024-12-20</mark> -  _23 pages, 10 figures. Accepted for publication in A&A on 26/11/2024_

</div>
<div id="authors">

L. H. Muñoz, et al. -- incl., <mark>F. Walter</mark>, <mark>T. Henning</mark>

</div>
<div id="abstract">

**Abstract:** Galaxy mergers are an important and complex phase during the evolution of galaxies. They may trigger nuclear activity and/or strong star forming episodes in the galaxy centres that potentially alter the evolution of the system. As part of the guaranteed time observations (GTO) program Mid-Infrared Characterization Of Nearby Iconic galaxy Centers (MICONIC), we used the medium-resolution spectrometer (MRS) of the Mid-Infrared Instrument (MIRI) on board of the James Webb Space Telescope (JWST) to study NGC 6240. We aim to characterise the dual active galactic nuclei (AGN), the ionised gas outflows and the main properties of the interstellar medium (ISM) over a mapped area of $6.6\arcsec \times 7.7\arcsec$ . We obtained integral field spectroscopic (IFS) mid-infrared data (wavelength from 4.9 to 28 $\mu$ m) of NGC 6240. We modelled the emission lines through a kinematic decomposition that accounts for the possible existence of various components. We have resolved both nuclei for the first time in the full 5-28 $\mu$ m spectral range. The fine-structure lines in the southern (S) nucleus are broader than for the northern (N) nucleus (full width at half maximum of $\geq$ 1500 vs $\sim$ 700 km s $^{-1}$ on average, respectively). High excitation lines, such as [ Ne V ] , [ Ne VI ] , and [ Mg V ] , are clearly detected in the N nucleus. In the S nucleus, the same lines can be detected but only after a decomposition of the PAH features in the integrated spectrum, due to a combination of a strong mid-IR continuum, the broad emission lines, and the intense star formation (SF). The SF is distributed all over the mapped FoV of 3.5 kpc $\times$ 4.1 kpc (projected), with the maximum located around the S nucleus. Both nuclear regions appear to be connected by a bridge region detected with all the emission lines. Based on the observed MRS line ratios and the high velocity dispersion ( $\sigma \sim 600$ km s $^{-1}$ ), shocks are also dominating the emission in this system. We detected the presence of outflows as a bubble north-west from the N nucleus and at the S nucleus. We estimated a ionised mass outflow rate of 1.4 $\pm 0.3$ M $_{\sun}$ yr $^{-1}$ and $1.8\pm0.2$ M $_{\sun}$ yr $^{-1}$ , respectively. Given the derived kinetic power of these outflows, both the AGN and the starburst could have triggered them.

</div>

<div id="div_fig1">

<img src="tmp_2412.14707/./figures/v_sigma_neii_1comp.png" alt="Fig8.1" width="25%"/><img src="tmp_2412.14707/./figures/v_sigma_neiii_1comp.png" alt="Fig8.2" width="25%"/><img src="tmp_2412.14707/./figures/v_sigma_feii_1comp.png" alt="Fig8.3" width="25%"/><img src="tmp_2412.14707/./figures/v_sigma_h1_65_1comp.png" alt="Fig8.4" width="25%"/>

**Figure 8. -** Kinematic maps obtained with the modelling of a single Gaussian for the [Ne II], [Ne III], [Fe II] at 5.34$\mu$m, and Pf$\alpha$ lines (from top to bottom, respectively). The latter line kinematics was obtained after re-binning the cube with a 2$\times$2 box (see Sect. \ref{Sect2:Methodology}). From left to right: flux in erg s$^{-1}$ cm$^{-2}$, velocity in km s$^{-1}$, and velocity dispersion in km s$^{-1}$. The contours in the velocity maps go from $-$300 to 300 km s$^{-1}$(increments of 100 km s$^{-1}$). The contours in the flux maps go from 10$^{-16}$ to 10$^{-14}$ erg s$^{-1}$ cm$^{-2}$(divided in 5 contours), except for Pf$\alpha$(from $10^{-17}$ to $10^{-15}$ erg s$^{-1}$ cm$^{-2}$). White stars indicate the photometric centre for both nuclei in their corresponding sub-channels, and the lower-left line indicate the 1$\arcsec$ physical scale. The (0,0) point in each panel marks the centre of the FoV at each channel. For all maps, north is up and east to the left. (*Fig:KinMapsLowExcit*)

</div>
<div id="div_fig2">

<img src="tmp_2412.14707/./figures/regions_NGC6240_v2.png" alt="Fig1" width="100%"/>

**Figure 1. -** Schematic figure of the main features seen in the flux and kinematic maps of the emission lines superimposed to the [Ne III] flux map contours (see Fig. \ref{Fig:KinMapsLowExcit}). In red we mark the position of the different detected clumps (namely C1, C2, and C3) and in green the extended emission regions (namely Ext1 and Ext2, see Sect. \ref{SubSect3:Results_fluxes}). The bubble-like structure mainly detected with the high-excitation lines is in blue (extending up to $\sim5.2$\arcsec$$, i.e. 2.74 kpc, see Fig. \ref{Fig:KinMapsHighIon} and Sect. \ref{SubSect4:Disc_Outflow}), and the bridge detected between both nuclei (separated $\sim 1.6$\arcsec$$, i.e. $\sim 840$ pc, see Sect. \ref{Subsect3:Results_IntProperties}) is marked in yellow. The purple regions indicate the "V"-shaped structure detected in the velocity dispersion maps. Additionally, we mark with black, dashed circles the region where we integrated the spectra for both nuclei, as well as their position with the white stars. (*Fig:SummaryRegions*)

</div>
<div id="div_fig3">

<img src="tmp_2412.14707/./figures/v_sigma_oiv_1comp.png" alt="Fig9.1" width="25%"/><img src="tmp_2412.14707/./figures/v_sigma_nev_1comp_isolated.png" alt="Fig9.2" width="25%"/><img src="tmp_2412.14707/./figures/v_sigma_nevi_1comp.png" alt="Fig9.3" width="25%"/><img src="tmp_2412.14707/./figures/v_sigma_mgv_1comp.png" alt="Fig9.4" width="25%"/>

**Figure 9. -** Kinematic maps obtained with the modelling of a single Gaussian for the [O IV], [Ne V], [Ne VI], and [Mg V] lines, top to bottom respectively. Full description in Fig. \ref{Fig:KinMapsLowExcit}. (*Fig:KinMapsHighIon*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2412.14707"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

290  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

10  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

1  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
